<a href="https://colab.research.google.com/github/remston-dsa/Proof-of-Work-Blockchain-using-Python-and-streamlit/blob/main/Copy_of_CUSP_Bitcoin_Code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install base58

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
pip install cryptography

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.0 MB 5.1 MB/s 


In [ ]:
pip install bitcoinaddress

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 142 kB 5.0 MB/s 


In [ ]:
pip install pycrypto 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 446 kB 5.0 MB/s 
  Created wheel for pycrypto: filename=pycrypto-2.6.1-cp37-cp37m-linux_x86_64.whl size=499932 sha256=e70f02ddbe8979034c44816c4f6c2a996bc7de9aeb5f9c292c692dc3ba47ee1c
  Stored in directory: /root/.cache/pip/wheels/cf/85/ba/bbd7c96add459de7598fb424e5ff2309baf2095c844ac0f191
Successfully built pycrypto


In [ ]:
pip install Crypto

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 590 kB 5.3 MB/s 


In [ ]:
import datetime
import time
import json
import hashlib
import codecs
import base58
from bitcoinaddress import Wallet
from Crypto.Hash import SHA256
from Crypto.Signature import PKCS1_v1_5

In [ ]:
class Owner:
  def __init__(self):
    self.privateKey, self.publicKey = self.generate_keys()
    self.bitcoinAddress = self.generate_bitcoin_address(self.publicKey)
    self.amount = 0

  def generate_keys(self):
    wallet = Wallet(testnet=True)
    return wallet.key.__dict__['testnet'].__dict__['wif'], wallet.address.pubkeyc

  def generate_bitcoin_address(self, publickey):
    publickey = codecs.decode(publickey, 'hex')
    s = hashlib.new('sha256', publickey).digest()
    r = hashlib.new('ripemd160', s).digest()
    return base58.b58encode(r).decode("utf-8")

  def balance(self):
    return self.amount

  def deposit(self, amount):
    self.amount += amount

  def withdraw(self, amount):
    if self.amount - amount > 0:
      self.amount -= amount

    else:
      print('Balance is Insufficient!')


In [ ]:
Remston = Owner()

In [ ]:
Renvil = Owner()

In [ ]:
class Transaction:
  def __init__(self, sender, recipient, amount):
    self.sender = sender
    self.recipient = recipient
    self.amount = amount
    self.time = datetime.datetime.now().strftime('%m/%d/%y, %H:%M:%S')
    self.hash = self.calculate_hash()
    self.txData = self.raw_tx()
    self.signature = self.sign()
    

  def calculate_hash(self):
    hashString = str(self.sender) + str(self.recipient) + str(self.amount) + str(self.time)
    hashEncode = json.dumps(hashString, sort_keys = True).encode()
    return hashlib.sha256(hashEncode).hexdigest()

  def is_valid(self):
    if self.sender == self.recipient:
      print('Sender and Recipient cannot be the same!')
      return False

    if self.hash != self.calculate_hash():
      print('Transaction has been tampered!') 
      return False

    return True

  def sign(self):
    txData = self.txData
    txHash = SHA256.new(str(txData).encode('utf8'))
    print(txHash)
    signer = PKCS1_v1_5.new(self.sender.privateKey)
    self.txData['Signature'] = signer.__dict__['_key']
    return self.txData['Signature']


  def raw_tx(self):
    return {
        'Sender' : self.sender,
        'Recipient' : self.recipient,
        'Amount' : self.amount,
        'Time' : self.time,
        'Hash' : self.hash,
        'Signature': None
    }

In [ ]:
tx = Transaction(Remston, Renvil , 10)

In [ ]:
tx.txData

{'Sender': <__main__.Owner at 0x7fe4b4a5c650>,
 'Recipient': <__main__.Owner at 0x7fe4b4a73b90>,
 'Amount': 10,
 'Time': '10/03/22, 23:56:54',
 'Hash': '5d8cc3a4eaa715feea855c78f35f7a150acd9707df4e4903d81c543c32c77572',
 'Signature': '92AD4XNHNYvUGwxTxbprmxtYSCGH9AoX4aFqfQ4MouUzj2MwGjK'}

In [ ]:
class Block:
  def __init__(self, transactions, index):
    self.index = index
    self.transactions = transactions
    self.timestamp = datetime.datetime.now().strftime('%m/%d/%y %H:%M:%S')
    self.nonce = 0
    self.prevHash = None
    self.hash = self.calculate_hash()
    self.Header = self.header()

  def calculate_hash(self):
    txString = ''
    for tx in self.transactions:
      txString += str(tx)

    hashString = str(txString) + str(self.index) + str(self.timestamp) +str(self.nonce) + str(self.prevHash)
    hashEncode = json.dumps(hashString, sort_keys = True).encode()
    return hashlib.sha256(hashEncode).hexdigest()

  def is_valid(self):
    for tx in self.transactions:
      if not tx.is_valid():
        print('Block is tampered!!')
        return False

    if self.hash != self.calculate_hash():
      print('Block is tampered!!')
      return False

    return True

  def mine(self, difficulty):
    if self.is_valid():
      hashPuzzle = ''.join([str(i) for i in range(difficulty)])

      while hashPuzzle != self.hash[:difficulty]:
        self.nonce += 1
        self.hash = self.calculate_hash()
        print(self.hash, self.nonce)
        #time.sleep(0.3)
      self.Header['Nonce'] = self.nonce
      self.Header['Hash'] = self.hash
      self.Header['Difficulty'] = difficulty
    else:
      return False

  def header(self):
    return {
        'Time' : self.timestamp,
        'Merkle Root' : None,
        'Prev Hash' :self.prevHash,
        'Hash' : self.hash,
        'Difficulty' : None,
        'Nonce': None
    }

In [ ]:
block = Block([tx], 0)

In [ ]:
block.mine(3)

203d3613d924256fea4c6d66461d10426ed41c15ee0d9434921fc48a959db5d5 1
abefd58f9f40c8a8223b80b56d62ada0231036a8835e8119d4d3cc0a9468bbf6 2
05d02bbf0e973d8c187fbdb63d5f0b11d6d29d6295a3185cdf207d0a15d6d9e2 3
6a81c5d9b8e285e7ab976b1776662885bf7baf59799ba86f9f70ec47a67e9447 4
5f9490ddb1e68a6f38f8ff6870941bc38f71f8c649eca3cab7246833781624bf 5
722c73425f1787fe5a159d9810f5062610b158ef4683ae1e928389e91a94e3d7 6
c0f0ad1c3521f78a954f0613212a6b4fa8f342442c8e1352dbf39e30056b599f 7
8eb7eb2858e9d7b2338a3291b688554b9e31862fce26a4e82b7139ab17d93693 8
69d84bee8df58958bc79d7f420d3ea2f7572526b58b25a4739553215bbd9933e 9
678006dfd529a0ada911b27b29235fc381afe34c3b0e2e88128ce4e3a9f7e322 10
a6dfffb5e62ebd6d388f8cf6967bd9cd12c42498650c737d510d891cb27c0d0f 11
85ac29d995e6b46e67ffd07ab29f6c6286ff21ceba018ac105efb62c96bbca7a 12
017caf383f8a6b72adc10bf611c022efe5e9986979ec8c1410b651934714721f 13
9fc2bc79018e40b3a5081f301fcea665d35d897502a5b4cd3d98088259665816 14
7e8210445844ca5cf5bca43b516c913c0be03ef2e0386701c794390d4

In [ ]:
block.Header

{'Time': '10/03/22 23:56:54',
 'Merkle Root': None,
 'Prev Hash': None,
 'Hash': '012ffe66e97967e9e0f75eb8a223c165cd9314c4663a00bbfaeb7d69e4707ce3',
 'Difficulty': 3,
 'Nonce': 1636}

In [ ]:
tx = [Transaction(Remston, Renvil , 10), Transaction(Renvil, Remston , 10)]

In [ ]:
class Blockchain:
  def __init__(self):
    self.owner = Owner()
    self.chain = []
    self.firstTx = Transaction(self.owner, Remston, 1000)
    self.pendingTransactions = []
    self.difficulty = 2
    self.blockSize = 10
    self.minerRewards = 50
    self.chain = self.create_genesis_block()
    

  def get_last_block(self):
    if len(self.chain) > 0:
      return self.chain[-1]

  def add_new_block(self, block):
    if len(self.chain) > 0:
      block.prevHash = self.get_last_block().hash
    else:
      block.prevHash = None

    self.chain.append(block)

  def create_genesis_block(self):
    self.pendingTransactions.append(self.firstTx)
    if self.mine_pending_tx(Remston):
      genesis_block = Block([self.firstTx], 0)
      return [genesis_block]

  def is_valid(self):
    for i in range(1, len(self.chain)):
      currBlock = self.chain[i]
      prevBlock = self.chain[i - 1]

      if currBlock.is_valid() and prevBlock.is_valid():
        print('Block is Tampered!!')
        return False

      if prevBlock.hash != currBlock.prevHash:
        print('Block is not part of the chain!!')
        return False

    return True

  def mine_pending_tx(self, miner):
    i = end = 0
    height = len(self.pendingTransactions)

    while end <= height:
      end = end + self.blockSize + i

      txSlice = [tx for tx in self.pendingTransactions[i: end]]
      print(txSlice)
      newBlock = Block(txSlice, len(self.chain))

      if newBlock.is_valid():
        newBlock.mine(self.difficulty)
        self.add_new_block(newBlock)
        rewards = Transaction(self.owner, miner, self.minerRewards)
        self.pendingTransactions = [rewards]
        return True
      return False

      i += 1

  def update_balance(self, node):
    for block in self.chain:
      for transaction in block.transactions:
        if transaction.sender == node:
          node.withdraw(transaction.amount)
        if transaction.recipient == node:
          node.deposit(transaction.amount)
    return node.balance

In [ ]:
Bitcoin = Blockchain()

ae21248b378b9500fda329e716a75d4bb6ee7f0b37a9ae057175ec02de1841ac 1
b50e958d2476f275c047d432804e390d28bf26b678756c5f6d3ea0a4d8d182a2 2
27dc9860f48086d47a284b753e37e0dcb2495661ab460fe8aa06e70e3b4aa49f 3
d9abc20e425f70f1de2799fb09e0fa576e35b23443daf69d7fd8499d92f77743 4
7652e96c0f58ed16686512849c1d95579537f0e7856ad77e1b9e12ebc3611d02 5
f08591cfe1863419e898c87579252f8c55dbe28ea13a61da037402b19cc8e203 6
4a1abe216e3fd2ed456de65138d33736e34e15bb8fd9bd09edc6d8a4d787fdd7 7
8168b2a4f07ea504157351d088c4fda6f53b1b874d2c273f6558b812e60a0919 8
f20bd525b2004eb4a2d3bb707c840c901e31a1acdff143554936dae601a88485 9
6390f0facf4cb9592d67e890749a6676c7ee4c34744cb19fbf51268b17eedb9b 10
ede678f8e1dc8d759e5559a8bde6c61d3cf29da0bfeaffd2094ded7d67f6318e 11
d86688470e91b34cfa549c6b02be34bb52c2683560af2550ae470403e6bfc13c 12
3a9625c71986d5f290da5e842f4f869d11378c2806a8f4e544b2cecd0bff510a 13
95dcce75a34d8a343bc5669c558c20e11360800d203712e4b7359bcef8795934 14
ff1859a88861ae5c836c55028436bbda16a8eb54be3b8eac32e68f697

In [ ]:
Bitcoin.chain[0].transactions[0].recipient

In [ ]:
Bitcoin.mine_pending_tx(Renvil)

5953240a2c7a55a285260a0729044e9218a3de3300ed6492edb117a640d64f53 1
feede41ffacc86ada9383c4c054c70d49d559b8b526f1dd9e348ccc3c8c4c14b 2
b357dd8955cbda845609c389fa9c9486d9e06e977221405aaf5c40089a13725c 3
2d2e95215b15bb37843c369f52dd53bcccc90b3aefa749a3895ce5cd5882d93c 4
f447e10e5412f4935980cd46da83cc0e3b7def4c6eadbb0894892d67e492da67 5
efc18ba24bf82fb57552c207c8bf123b5343f0177dae85fffd18e6f53f12baad 6
bb1d305ce2fc2332aef17693c029c93979b4e2a9071eaa1a28c9099a8ea987a6 7
3b0c2f7321ec731d5a5a121499ac33d5eb3ca93ec33e6ac661fa5c867426e278 8
489c225e950e03c7f618f11c497a4a3311b810e5c2c6abfa03daccf484a499d2 9
323a738e063132c89e4b28ff5a912378d892582541967b11e27859bc7e7a5532 10
25e871f8bddf4038e2d405aa352ac890e0d5891842946995fb0d2dd6420f012f 11
20376efae1fd23ade9ede6465e95a1a11e8aede1fba3b34ea7b29c0bf6b326eb 12
befc726cf181ca9fc44cca3af23570352f061e8fa60d8595624cd0d239b5093f 13
a1ef3c4fb108534291ba4f32e4a599bf444e1ddd95c1add00e7c42182eb08a82 14
f2dffeede0fa926e0dc8a356d4822e9da0bec14d128b698d16697f03b

True

In [ ]:
Bitcoin.update_balance(Renvil)

<bound method Owner.balance of <__main__.Owner object at 0x7fe4b4a73b90>>

In [ ]:
Remston.balance()

0

In [ ]:
Renvil.balance()

0

In [ ]:
Bitcoin.pendingTransactions

In [ ]:
Remston.bitcoinAddress

'6Dun5Nw1F163uA62t5JkyN1gkpe'

In [ ]:
Renvil.bitcoinAddress

'4NW4KT8BgAQ8cpkRLG1GoM32s3y4'

In [ ]:
l = [1, 2, 3]

In [ ]:
l[:10]

[1, 2, 3]

In [ ]:
pip install bitcoin

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from bitcoin import *
my_private_key = '91d6gaFk1pjYK8WyK8acfExxzR1C3RhjMxyaK8CygHh5mAuFDpV'
my_public_key = privtopub(my_private_key)
my_bitcoin_address = pubtoaddr('0462fd3b239a42d276238d65491977199bfa42b3809278a11f74bef22aa746d20fa102500063a221a7b203965d66069b6ef4030b71e53ac6916975c773ce61e7a2')
print(my_bitcoin_address)

1C5JoxQ2HjE2dtcLxQ4oKpq1wFcY6oge4C


In [ ]:
print(history('12KKDt4Mj7N5UAkQMN7LtPZMayenXHa8KL'))

Fetching more transactions... 50
Fetching more transactions... 100
Fetching more transactions... 150
Fetching more transactions... 200
Fetching more transactions... 250
Fetching more transactions... 300
Fetching more transactions... 350
Fetching more transactions... 400
Fetching more transactions... 450
Fetching more transactions... 500
Fetching more transactions... 550
Fetching more transactions... 600
Fetching more transactions... 650
Fetching more transactions... 700


[{'address': '12KKDt4Mj7N5UAkQMN7LtPZMayenXHa8KL', 'value': 634292298, 'output': '1c42d00ea52f50be66596552cedd8ddf4b9270b59b66eb7df9c3bf10f1aa8087:0', 'block_height': 756933}, {'address': '12KKDt4Mj7N5UAkQMN7LtPZMayenXHa8KL', 'value': 638005827, 'output': '9433da6a1324aab9ab5db651481bcbc86f0b51c44bc0b7a56c0433caa8735e49:0', 'block_height': 756932}, {'address': '12KKDt4Mj7N5UAkQMN7LtPZMayenXHa8KL', 'value': 629613225, 'output': 'cf05330ee3f690c3397535b29f7204d0d29490bde50f8986e3b9abbb8c84ebdd:0', 'block_height': 756931}, {'address': '12KKDt4Mj7N5UAkQMN7LtPZMayenXHa8KL', 'value': 628510355, 'output': '329a30847dac4a0979476cb5d055ffa50c172e2526f26b9ec5aaf4b58d540c2f:0', 'block_height': 756928}, {'address': '12KKDt4Mj7N5UAkQMN7LtPZMayenXHa8KL', 'value': 633864815, 'output': 'eafa317c524e6f48775e10d465fa965ba840b12daf4fbd3fc39a52e931233590:0', 'block_height': 756927}, {'address': '12KKDt4Mj7N5UAkQMN7LtPZMayenXHa8KL', 'value': 626182832, 'output': '3c1d4ccc7aa353d0249e82501254bf05f6fa8b49427